# Create database

### Connection

In [1]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"
NAME_DB = "neo4j"

driver = GraphDatabase.driver(URI)

driver.verify_connectivity()
print("Successfully connected to Neo4j")

session = driver.session(database="system")

if [NAME_DB] in session.run("SHOW DATABASE yield name;").values():
    print("Database already exists")
else:
    session.run(f"CREATE DATABASE {NAME_DB};")
    print("Database created")
session = driver.session(database=NAME_DB)

Successfully connected to Neo4j
Database already exists


### Create constraints
First of all it adds property uniqueness constraints for all nodes.  
For this study is not so much useful other constraints because dataset was checked in preprocessing and there will be other import of data.  
If you want other constraints you can visit Cypher manual [here](https://neo4j.com/docs/cypher-manual/current/constraints/syntax/)

In [2]:
# CONSTRAINTS for unique IDs
session.run(
    """
    CREATE CONSTRAINT UniqueMovieID IF NOT EXISTS 
    FOR (m:Movie) 
    REQUIRE m.id IS unique;
    """
)
session.run(
    """
    CREATE CONSTRAINT UniqueUserID IF NOT EXISTS 
    FOR (u:User) 
    REQUIRE u.id IS unique;
    """
)
session.run(
    """
    CREATE CONSTRAINT UniqueGenreID IF NOT EXISTS 
    FOR (g:Genre) 
    REQUIRE g.id IS unique;
    """
)

## Add nodes

In [2]:
session.run(
    """
    LOAD CSV WITH HEADERS 
    FROM "file:///users.csv" AS row
    MERGE (u:User {id: toInteger(row.userId)})
    SET u.gender = row.gender,
        u.age = toInteger(row.age),
        u.occupation = toInteger(row.occupation);
    """
)

session.run(
    """
    LOAD CSV WITH HEADERS 
    FROM "file:///movies.csv" AS row
    MERGE (m:Movie {id: toInteger(row.movieId)})
    SET m.title = row.title,
        m.year = toInteger(row.year);
    """
)

session.run(
    """
    LOAD CSV WITH HEADERS 
    FROM "file:///genres.csv" AS row
    MERGE (g:Genre {id: toInteger(row.genreId)})
    SET g.name = row.name;
    """
)

## Create useful indexes
After the creation of constraints, in the server be created ranged index for ids and lookup index

In [ ]:
session.run(
    """
    CREATE INDEX RangeIndexTimestampRatings FOR ()-[r:RATED]-() ON (r.timestamp)
    """
)

## Add relationships

In [ ]:
session.run(
    """
    LOAD CSV WITH HEADERS FROM "file:///ratings.csv" AS row
    CALL {
        WITH row
        MATCH (m:Movie {id: toInteger(row.movieId)})
        MATCH (u:User {id: toInteger(row.userId)})
        MERGE (u)-[r:RATED]->(m)
        SET r.rating = toInteger(row.rating),
        r.timestamp = datetime(row.timestamp)
    } IN TRANSACTIONS OF 10000 ROWS;

    """
)

session.run(
    """
    LOAD CSV WITH HEADERS FROM "file:///movies_genres.csv" AS row
    MATCH (m:Movie {id: toInteger(row.movieId)})
    MATCH (g:Genre {id: toInteger(row.genreId)})
    MERGE (m)-[in_g:IN_GENRE]->(g);

    """
)